# Transposing data of genre, actors, directors and writers
### Why? - To simplify the work in Tableau

In [12]:
# import all packages
import pandas as pd
import numpy as np
import psycopg2 as psycopg2
import sql_functions as sqlf
import string as string

In [13]:
# pd.set_option("display.max.columns", 264)

In [14]:
schema = "capstone_24_4_group1"

In [15]:
imdb_query = f'''   SELECT *
                    FROM {schema}."IMDB_data"
                    '''

In [16]:
imdb_df = sqlf.get_dataframe(imdb_query)

In [17]:
imdb_df.head(3)

,tconst,primary_title,original_title,year,runtime,num_votes,average_rating,genres_count,genre,genre2,genre3,acting,acting2,acting3,acting4,acting5,directors_count,director_name,director2_name,director3_name,writers_count,writer_name,writer2_name,writer3_name,is_adult
0,tt0013274,Istoriya grazhdanskoy voyny,Istoriya grazhdanskoy voyny,2021,94,73.0,6.7,1.0,Documentary,None,None,None,None,None,None,None,2.0,Nikolai Izvolov,Dziga Vertov,None,NaN,None,None,None,0
1,tt0015414,La tierra de los toros,La tierra de los toros,2000,60,17.0,5.4,NaN,None,None,None,None,None,None,None,None,1.0,Musidora,None,None,NaN,None,None,None,0
2,tt0035423,Kate & Leopold,Kate & Leopold,2001,118,89944.0,6.4,3.0,Comedy,Fantasy,Romance,Meg Ryan,Hugh Jackman,Liev Schreiber,Breckin Meyer,Natasha Lyonne,1.0,James Mangold,None,None,2.0,Steven Rogers,James Mangold,None,0


## Transposing actors

In [34]:
imdb_actors = imdb_df[["tconst", "acting", "acting2", "acting3", "acting4", "acting5"]]

df_melted_actors = pd.melt(imdb_actors, id_vars=['tconst'], value_vars=["acting", "acting2", "acting3", "acting4", "acting5"],
                    var_name='Spalte', value_name='actor')

df_melted_actors = df_melted_actors[["tconst","actor"]]
df_melted_actors.head(3)

df_actors = df_melted_actors

## Transposing genres

In [48]:
imdb_genres = imdb_df[["tconst", "genre",	"genre2", "genre3"]]

df_melted_genres = pd.melt(imdb_genres, id_vars=['tconst'], value_vars=["genre", "genre2", "genre3"],
                    var_name='Spalte', value_name='Genre')

df_melted_genres = df_melted_genres[["tconst","Genre"]]
df_melted_genres.head(3)

df_genres = df_melted_genres

df_genres.columns = df_genres.columns.str.lower()

## Transposing directors

In [49]:
imdb_directors = imdb_df[["tconst", "director_name", "director2_name", "director3_name"]]

df_melted_directors = pd.melt(imdb_directors, id_vars=['tconst'], value_vars=["director_name", "director2_name", "director3_name"],
                    var_name='Spalte', value_name='Director')

df_melted_directors = df_melted_directors[["tconst","Director"]]
df_melted_directors.head(3)

df_directors = df_melted_directors

df_directors.columns = df_directors.columns.str.lower()

## Transposing writers

In [50]:
imdb_writers = imdb_df[["tconst", "writer_name", "writer2_name", "writer3_name"]]

df_melted_writers = pd.melt(imdb_writers, id_vars=['tconst'], value_vars=["writer_name", "writer2_name", "writer3_name"],
                    var_name='Spalte', value_name='Writer')

df_melted_writers = df_melted_writers[["tconst","Writer"]]
df_melted_writers.head(3)

df_writers = df_melted_writers

df_writers.columns = df_writers.columns.str.lower()

## Display all to check

In [ ]:
display(df_actors)
display(df_genres)
display(df_directors)
display(df_writers)

## Upload

In [53]:
schema

'capstone_24_4_group1'

In [55]:
dataframes = {
    "actors":df_actors,
    "genres":df_genres,
    "directors":df_directors,
    "writers":df_writers,
}

In [59]:
# table_name = 'eu_merged'
engine = sqlf.get_engine()
engine
# Write records stored in a dataframe to SQL database
if engine != None:
    try:
        for table_name, df in dataframes.items():
            df.to_sql(table_name, # Name of SQL table
                            con=engine, # Engine or connection
                            if_exists='replace', # Drop the table before inserting new values 
                            schema=schema, # your class schema
                            index=False, # Write DataFrame index as a column
                            chunksize=5000, # Specify the number of rows in each batch to be written at a time
                            method='multi') # Pass multiple values in a single INSERT clause
            print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None
else:
    print("nope") 

The actors table was imported successfully.
The genres table was imported successfully.
The directors table was imported successfully.
The writers table was imported successfully.
